In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt

import numpy as np
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

from Data import Data

In [2]:
data = Data()
samples = data.getSamples()
targets = data.getTargets()

minimum = np.array(samples, dtype=np.float64).min(axis=0).min(axis=0)
maximum = np.array(samples, dtype=np.float64).max(axis=0).max(axis=0)
print("Maximum: " + str(maximum))
print("Minimum: " + str(minimum))

Maximum: 54077760.0
Minimum: 0.0


In [3]:
targetsGroup = []
for target in targets:
    if target not in targetsGroup:
        targetsGroup.append(target)

groupedSamples = [[] for i in range(len(targetsGroup))]
means = []
for i in range(len(targetsGroup)):
    for j in range(len(samples)):
        if targets[j] == i + 1:
            groupedSamples[i].append(np.array(samples[j]))
    groupedSamples[i] = np.array(groupedSamples[i])

for group in groupedSamples:
    group2 = np.array(group, dtype=np.float64)
    mean = []
    for i in range(group2.shape[1]):
        a = group2[:, i - 1]
        a = a.mean()
        mean.append(a)
    means.append(mean)

dataFrame = pd.DataFrame(means)

usersRangeWidget = widgets.IntRangeSlider(
    value=[1, 6],
    min=min(targetsGroup),
    max=max(targetsGroup),
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
usersRangeWidget.layout.width = '100%'

@interact(x=usersRangeWidget)
def updatePlots(x):
    meansPlot = dataFrame.iloc[x[0]:x[1]].T
    meansPlot.plot(legend=False).set_ylim(dataFrame.min(axis=0).min(axis=0), dataFrame.max(axis=0).max(axis=0))

interactive(children=(IntRangeSlider(value=(1, 6), continuous_update=False, description='Test:', layout=Layout…

In [4]:
usersDDWidget = widgets.Dropdown(
    options=targetsGroup,
    value=1,
    description='User:',
    disabled=False,
)

@interact(x=usersDDWidget)
def plotUserData(x):
    usersSamples = pd.DataFrame(groupedSamples[x])
    usersSamples.astype(float).T.plot(legend=False).set_ylim(minimum, maximum)


interactive(children=(Dropdown(description='User:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15…

## Plotting the features

Here we plot one feature against another to quickly see if users can be discriminated (i.e. if we can separate the groups of points with a straight line or if a grouping is possible).

Extending this feature plotting to a third dimension (by adding another feature) can help discriminating the users. By extension, we can assume that up to it's 60th dimension, the points can be clustered and separated.

In [5]:
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

users = data.table["user_id"].drop_duplicates()

firstUserDropdownWidget = widgets.Dropdown(options=users, value=1, description='First user:')
secondUserDropdownWidget = widgets.Dropdown(options=users, value=2, description='Second user:')

fig2d = plt.figure()
fig3d = plt.figure()

ax2d = fig2d.add_subplot(111)
ax2d.set_xlabel('rrTime0')
ax2d.set_ylabel('rrTime1')

ax3d = fig3d.add_subplot(111, projection='3d')
ax3d.set_xlabel('rrTime0')
ax3d.set_ylabel('rrTime1')
ax3d.set_zlabel('rrTime2')

@interact(x=firstUserDropdownWidget, y=secondUserDropdownWidget)
def plotFeatures(x, y):
    sub = data.table.loc[(data.table["user_id"] == x) | (data.table["user_id"] == y),
                        ["user_id", "rrTime0", "rrTime1", "rrTime2"]]
    ax2d.scatter(sub["rrTime0"], sub["rrTime1"], c=sub["user_id"])
    ax3d.scatter(sub["rrTime0"], sub["rrTime1"], sub["rrTime2"], c=sub["user_id"])

FigureCanvasNbAgg()

FigureCanvasNbAgg()

interactive(children=(Dropdown(description='First user:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …